# $$ Part 01 $$

In [ ]:
!pip install patchify

In [ ]:
import os
import cv2
from PIL import Image 
import numpy as np 
from patchify import patchify
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from matplotlib import pyplot as plt
import random

In [ ]:
minmaxscaler = MinMaxScaler()

In [1]:
!ls -lah 'D:/GitHub/FinalYearProject/Week 004/Usman Ghias/Part 01'

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
dataset_name = "DubaiDataset"

In [ ]:
# This code uses the os.walk function to traverse through a directory structure.
# It explores all subdirectories and files in the specified dataset folder.

for path, subdirs, files in os.walk(os.path.join(dataset_root_folder, dataset_name)):
    # For each directory (path) in the dataset, extract the directory name.
    dir_name = path.split(os.path.sep)[-1]

    # Check if the current directory is named 'masks'.
    if dir_name == 'masks':
        # If it is the 'masks' directory, list all the files (images) in that directory.
        images = os.listdir(path)
        print(path)  # Print the path to the 'masks' directory.

        # Iterate through the list of image files.
        for i, image_name in enumerate(images):
            # Check if the file has a '.png' extension.
            if image_name.endswith('.png'):
                # If the file has a '.png' extension, set the variable 'a' to True.
                a = True  # You might use this variable for some further processing.

In [ ]:
# image_patch_size to 256, likely indicating the size (in pixels) for image patches in some image processing task
image_patch_size = 256

In [ ]:
# Load an image using OpenCV (cv2).
# The image is loaded from the specified path, which is constructed from 'dataset_root_folder' and 'dataset_name'.
# The '1' parameter indicates that the image should be loaded in color (with channels).
image = cv2.imread(f'{dataset_root_folder}/{dataset_name}/Tile 2/images/image_part_001.jpg', 1)


In [ ]:
image.shape

In [ ]:
# Create image patches using the 'patchify' function with the specified 'image_patch_size' and step size.
# 'image' is the input image for patch extraction.
# (image_patch_size, image_patch_size, 3) defines the patch size and the number of color channels (3 for color images).
# 'step' parameter controls the separation between patches.
image_patches = patchify(image, (image_patch_size, image_patch_size, 3), step=image_patch_size)

In [ ]:
len(image_patches)

In [ ]:
print(image_patches.shape)

In [ ]:
minmaxscaler = MinMaxScaler()
# A Min-Max Scaler is a data preprocessing technique used to scale numeric data to a specific range, often between 0 and 1, 
# by shifting and scaling the values. It's commonly used in data preprocessing for machine learning and data analysis tasks.

In [ ]:
# Check the shape of the first row (index 0) in 'image_y'.
image_y[0].shape

In [ ]:
# Calculate the nearest multiple of 'image_patch_size' that is less than or equal to 'image.shape[0]'.
(image.shape[0] // image_patch_size) * image_patch_size

In [ ]:
type(Image.fromarray(image))

In [ ]:
# Create an image object from the 'image' array and then check its data type using 'type'.
type(Image.fromarray(image))

In [ ]:
# Check and return the shape of the 'image' array.
image.shape

In [ ]:
# Calculate the nearest multiple of 'image_patch_size' that is less than or equal to 'image.shape[0]'.
(image.shape[0] // image_patch_size) * image_patch_size

In [ ]:
# Initialize empty lists to store image and mask datasets.
image_dataset = []
mask_dataset = []

# Iterate through two types: 'images' and 'masks'.
for image_type in ['images', 'masks']:
    
    # Define the image file extension based on the image type.
    if image_type == 'images':
        image_extension = 'jpg'
    elif image_type == 'masks':
        image_extension = 'png'
    
    # Loop through tile IDs (1 to 7).
    for tile_id in range(1, 8):
        
        # Loop through image IDs (1 to 19).
        for image_id in range(1, 20):
            
            # Load an image using OpenCV based on the specified path.
            image = cv2.imread(f'{dataset_root_folder}/{dataset_name}/Tile {tile_id}/{image_type}/image_part_00{image_id}.{image_extension}', 1)
            
            # Check if the image was loaded successfully.
            if image is not None:
                
                # If the image type is 'masks,' convert it to RGB format.
                if image_type == 'masks':
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                
                # Calculate the size (in pixels) for cropping the image.
                size_x = (image.shape[1] // image_patch_size) * image_patch_size
                size_y = (image.shape[0] // image_patch_size) * image_patch_size
                
                # Create an image object from the array, crop it, and convert it back to an array.
                image = Image.fromarray(image)
                image = image.crop((0, 0, size_x, size_y))
                image = np.array(image)
                
                # Create patches from the image.
                patched_images = patchify(image, (image_patch_size, image_patch_size, 3), step=image_patch_size)
                
                # Loop through the patches.
                for i in range(patched_images.shape[0]):
                    for j in range(patched_images.shape[1]):
                        if image_type == 'images':
                            # If the image type is 'images,' preprocess the individual patch.
                            individual_patched_image = patched_images[i, j, :, :]
                            individual_patched_image = minmaxscaler.fit_transform(individual_patched_image.reshape(-1, individual_patched_image.shape[-1])).reshape(individual_patched_image.shape)
                            individual_patched_image = individual_patched_image[0]
                            image_dataset.append(individual_patched_image)
                        elif image_type == 'masks':
                            # If the image type is 'masks,' extract the individual mask patch.
                            individual_patched_mask = patched_images[i, j, :, :]
                            individual_patched_mask = individual_patched_mask[0]
                            mask_dataset.append(individual_patched_mask)


In [ ]:
# Print the number of elements in the 'image_dataset' and 'mask_dataset' lists.
print(len(image_dataset))  # Number of elements in the 'image_dataset'.
print(len(mask_dataset))   # Number of elements in the 'mask_dataset'.

In [ ]:
# Convert the 'image_dataset' and 'mask_dataset' lists into NumPy arrays.
image_dataset = np.array(image_dataset)  # Convert 'image_dataset' into a NumPy array.
mask_dataset = np.array(mask_dataset)    # Convert 'mask_dataset' into a NumPy array.

In [ ]:
# Print the number of elements in the 'image_dataset' and 'mask_dataset' NumPy arrays.
print(len(image_dataset))  # Number of elements in the 'image_dataset' NumPy array.
print(len(mask_dataset))   # Number of elements in the 'mask_dataset' NumPy array.

In [ ]:
# Check and print the data type of the first element in the 'image_dataset' array.
type(image_dataset[0])

In [ ]:
# Check and print the data type of the NumPy array obtained by reshaping the first element of 'image_dataset'.
type(np.reshape(image_dataset[0], (image_patch_size, image_patch_size, 3)))

In [ ]:
# Generate a random image ID within the range of available images.
random_image_id = random.randint(0, len(image_dataset))

# Create a matplotlib figure to display two subplots side by side.
plt.figure(figsize=(14, 8))

# Define the first subplot (on the left) to display a random image from 'image_dataset'.
plt.subplot(121)
plt.imshow(image_dataset[random_image_id])  # Display the image.

# Define the second subplot (on the right) to display the corresponding mask from 'mask_dataset'.
plt.subplot(122)
plt.imshow(mask_dataset[random_image_id])   # Display the mask.

In [ ]:
class_building = '#3C1098'
class_building = class_building.lstrip('#')
class_building = np.array(tuple(int(class_building[i:i+2], 16) for i in (0,2,4)))
print(class_building)

class_land = '#8429F6'
class_land = class_land.lstrip('#')
class_land = np.array(tuple(int(class_land[i:i+2], 16) for i in (0,2,4)))
print(class_land)

class_road = '#6EC1E4'
class_road = class_road.lstrip('#')
class_road = np.array(tuple(int(class_road[i:i+2], 16) for i in (0,2,4)))
print(class_road)

class_vegetation = '#FEDD3A'
class_vegetation = class_vegetation.lstrip('#')
class_vegetation = np.array(tuple(int(class_vegetation[i:i+2], 16) for i in (0,2,4)))
print(class_vegetation)

class_water = '#E2A929'
class_water = class_water.lstrip('#')
class_water = np.array(tuple(int(class_water[i:i+2], 16) for i in (0,2,4)))
print(class_water)

class_unlabeled = '#9B9B9B'
class_unlabeled = class_unlabeled.lstrip('#')
class_unlabeled = np.array(tuple(int(class_unlabeled[i:i+2], 16) for i in (0,2,4)))
print(class_unlabeled)

In [ ]:
# Obtain the number of elements (patches) in the first dimension of the 'mask_dataset' NumPy array.
mask_dataset.shape[0]

In [ ]:
# Assign the content of the 'individual_patched_mask' variable to a new variable 'label'.
label = individual_patched_mask

In [ ]:
# Define a function 'rgb_to_label' to convert an RGB label into a single-channel label segment.
def rgb_to_label(label):
    # Initialize a new label segment array of the same shape as the input 'label.'
    label_segment = np.zeros(label.shape, dtype=np.uint8)

    # For each class color (e.g., class_water, class_land), assign a numeric label to 'label_segment.'
    # Numeric labels are based on class colors, where 0 represents 'class_water,' 1 represents 'class_land,' and so on.
    label_segment[np.all(label == class_water, axis=-1)] = 0
    label_segment[np.all(label == class_land, axis=-1)] = 1
    label_segment[np.all(label == class_road, axis=-1)] = 2
    label_segment[np.all(label == class_building, axis=-1)] = 3
    label_segment[np.all(label == class_vegetation, axis=-1)] = 4
    label_segment[np.all(label == class_unlabeled, axis=-1)] = 5

    # Remove the color channels, leaving a single-channel label segment.
    label_segment = label_segment[:, :, 0]

    # Return the resulting label segment.
    return label_segment

In [ ]:
# Initialize an empty list 'labels' to store label segments for each mask patch.
labels = []

# Loop through each mask patch in the 'mask_dataset.'
for i in range(mask_dataset.shape[0]):
    # Call the 'rgb_to_label' function to convert an RGB mask into a single-channel label segment.
    label = rgb_to_label(mask_dataset[i])
    
    # Append the resulting label segment to the 'labels' list.
    labels.append(label)


In [ ]:
# Print the number of elements (label segments) in the 'labels' list.
print(len(labels))

In [ ]:
# Convert the 'labels' list, which contains label segments, into a NumPy array.
labels = np.array(labels)

In [ ]:
# Access and retrieve the label segment at index 3 in the 'labels' NumPy array.
labels[3]

In [ ]:
# Expand the dimensions of the 'labels' NumPy array by adding a new axis along the third dimension.
labels = np.expand_dims(labels, axis=3)

In [ ]:
# Access and retrieve the label segment at index 0 in the 'labels' NumPy array.
labels[0]


In [ ]:
# Retrieve the unique values present in the 'labels' NumPy array.
np.unique(labels)


In [ ]:
# Print the total unique labels based on masks by formatting the result.
print("Total unique labels based on masks:", format(np.unique(labels)))

In [ ]:
# Generate a random image ID within the range of available images.
random_image_id = random.randint(0, len(image_dataset))

# Create a matplotlib figure to display two subplots side by side.
plt.figure(figsize=(14, 8))

# Define the first subplot (on the left) to display a random image from 'image_dataset.'
plt.subplot(121)
plt.imshow(image_dataset[random_image_id])  # Display the image.

# Define the second subplot (on the right) to display the corresponding label segment from 'labels.'
# The label segment is displayed using the grayscale colormap (single channel).
plt.subplot(122)
plt.imshow(labels[random_image_id][:, :, 0])  # Display the label segment.


In [ ]:
# Access and retrieve the first label segment's content as a single-channel array from 'labels.'
labels[0][:, :, 0]

In [ ]:
# Calculate the total number of unique classes (labels) present in the 'labels' array.
total_classes = len(np.unique(labels))


In [ ]:
# Retrieve and return the total number of unique classes (labels) in the dataset.
total_classes


In [ ]:
# Import the 'to_categorical' function from the TensorFlow module 'keras.utils.'
from tensorflow.keras.utils import to_categorical

In [ ]:
# Convert the 'labels' array into a categorical representation using one-hot encoding.
# 'num_classes' specifies the total number of unique classes.
labels_categorical_dataset = to_categorical(labels, num_classes=total_classes)


In [ ]:
# Retrieve and return the shape of the 'labels_categorical_dataset' NumPy array.
labels_categorical_dataset.shape


In [ ]:
# Assign the 'image_dataset' to the 'master_training_dataset' variable.
master_training_dataset = image_dataset


In [ ]:
# Import the 'train_test_split' function from the scikit-learn library for data splitting.
from sklearn.model_selection import train_test_split


In [ ]:
# Split the training dataset and corresponding labels into training and testing subsets.
# 'master_trianing_dataset' contains the input data, and 'labels_categorical_dataset' contains the target labels.
# 'test_size' specifies the proportion of data to allocate to the testing subset.
# 'random_state' sets the random seed for reproducibility.

X_train, X_test, y_train, y_test = train_test_split(master_trianing_dataset, labels_categorical_dataset, test_size=0.15, random_state=100)


In [ ]:
# Print the shapes (dimensions) of the training and testing subsets for both data and labels.
print(X_train.shape)  # Shape of the training data (X_train).
print(X_test.shape)   # Shape of the testing data (X_test).
print(y_train.shape)  # Shape of the training labels (y_train).
print(y_test.shape)   # Shape of the testing labels (y_test).


In [ ]:
# Calculate and assign values to variables representing image dimensions and total classes.

# 'image_height' stores the height (number of rows) of the images in the training dataset.
image_height = X_train.shape[1]

# 'image_width' stores the width (number of columns) of the images in the training dataset.
image_width = X_train.shape[2]

# 'image_channels' represents the number of color channels in the images (e.g., 3 for RGB images).
image_channels = X_train.shape[3]

# 'total_classes' holds the total number of unique classes in the training labels.
total_classes = y_train.shape[3]


In [ ]:
# Print the values of variables representing image dimensions and total classes.

print(image_height)    # Print the height of the images in the training dataset.
print(image_width)     # Print the width of the images in the training dataset.
print(image_channels)  # Print the number of color channels in the images.
print(total_classes)   # Print the total number of unique classes in the training labels.


# $$ Usman Ghias - The END - Part 01 $$